# Simple Genetic Algorithm

In [ ]:
using Pkg
Pkg.activate("/home/lism8025/Documents/Projects/GeneticDeorbit")

In [ ]:
using Distributed 
@everywhere using GeneticDeorbit
@everywhere using Evolutionary
@everywhere import Evolutionary: trace!
@everywhere using Distributions, Random
using Plots

In [ ]:
# custom result trace
function trace!(record::Dict{String,Any}, objfun, state, population, method, options)
    record["state"] = deepcopy(state)
    record["objfun"] = deepcopy(objfun)
    record["population"] = deepcopy(population)
end

In [ ]:
function rand_x0() 
    return [rand(Uniform(lower[1], upper[1])), rand(Uniform(lower[2], upper[2]))]
end

# selection function picks top N of population
function myselection
end

# crossover function averages values
function mycrossover(v1::T, v2::T; rng::AbstractRNG=Random.default_rng()) where {T <: AbstractVector}
    c1 = (v1+v2)./2
    return c1, copy(c1)
end

# mutation function sometimes creates new random offspring
function mymutation(v1::T; rng::AbstractRNG=Random.default_rng()) where {T <: AbstractVector}
    return rand_x0()
end

In [ ]:
# Params
normalizer = [5184000, 5230]
weights = [1.0, 0.0]./normalizer # 0% on throttle
lower = [250.0e3, 0.0]
upper = [450.0e3, pi/2]

# Optimize
result = Evolutionary.optimize(
         x-> eval_fitness_val(x,weights),
         BoxConstraints(lower, upper),
         rand_x0,
         GA(populationSize = 20, 
            selection = rouletteinv,
            crossover = AX, 
            mutation = mymutation,
            epsilon = 4
            ),
         Evolutionary.Options(iterations=100,
                              time_limit=8*60*60.0,
                              store_trace = true,
                              show_trace = false,
                              parallelization=:thread, # :serial, :thread
                              )
         ) 

In [ ]:
gene_hist1 = []
gene_hist2 = []
avg_fit_hist = []
best_fit_hist = []
best_gene1 = []
best_gene2 = []
pop_fit_hist = []
for iteration in result.trace
    population = permutedims(hcat(iteration.metadata["population"]...))
    gene1 = population[:,1]
    gene2 = population[:,2]
    avg_fit = mean(iteration.metadata["state"].fitpop)
    push!(pop_fit_hist, iteration.metadata["state"].fitpop)
    push!(avg_fit_hist, avg_fit)
    push!(best_fit_hist, iteration.value)
    push!(gene_hist1, gene1)
    push!(best_gene1, iteration.metadata["state"].fittest[1])
    push!(gene_hist2, gene2)
    push!(best_gene2, iteration.metadata["state"].fittest[2])
end

In [ ]:
opt_sc = spacecraft(result.minimizer[1], result.minimizer[2])
opt_traj = eval_fitness(opt_sc, [1.0, 1.0]./[5184000,5230])
plotOptimal(opt_traj, gene_hist1, gene_hist2)

In [ ]:
splt = scatter(ones(size(gene_hist1[1])), gene_hist1[1])
for i = 2:length(gene_hist1)
    scatter!(splt, i.*ones(size(gene_hist1[i])), gene_hist1[i])
end
plot!(splt, best_gene1, markershape=:star, label="Iteration Best")

@show splt

In [ ]:
splt2 = scatter(ones(size(gene_hist2[1])), gene_hist2[1]./(pi))
for i = 2:length(gene_hist2)
    scatter!(splt2, i.*ones(size(gene_hist2[i])), gene_hist2[i]./(pi))
end
plot!(splt2, best_gene2/(pi), markershape=:star, label="Iteration Best")

@show splt2

In [ ]:
plt = plot(avg_fit_hist, markershape=:^, label="Iteration Average")
plot!(plt, best_fit_hist, markershape=:star, label="Iteration Best")

for i = 1:length(pop_fit_hist)
    scatter!(plt, i.*ones(size(pop_fit_hist[i])), pop_fit_hist[i], label="Iteration " * string(i), legend=:topright)
end

@show plt

In [ ]:
using JSON
# Save Results
save_dict = Dict("weights" => weights, 
                 "gene1_history" => hcat(gene_hist1...), 
                 "gene2_history" => hcat(gene_hist2...),
                 "pop_fit_hist"  => hcat(pop_fit_hist...),
                 "fittest_hist"  => hcat(best_gene1, best_gene2),
                 "fittest_val_hist"  => best_fit_hist,
                 )

open("throttle0.json","w") do f
  JSON.print(f, save_dict, 4)
end

In [ ]:
using Pkg
Pkg.activate("/home/lism8025/Documents/Projects/GeneticDeorbit")
using GeneticDeorbit
using JSON, Plots
data = JSON.parsefile("throttle0.json")
opt_sc = spacecraft(data["fittest_hist"][1][end],  data["fittest_hist"][2][end])
opt_traj = eval_fitness(opt_sc, [1.0, 1.0]./[1.248e7,5230])
plotOptimal(opt_traj, data["gene1_history"], data["gene2_history"])
savefig("throttle0.png")